In [25]:
import folium
import pandas as pd

In [26]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [27]:
import requests
import pandas as pd
import io

# Define the URL
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the CSV file from the URL
response = requests.get(URL)
response.raise_for_status()  # Ensure the request was successful

# Read the CSV file into a pandas DataFrame
spacex_csv_file = io.StringIO(response.text)
spacex_df = pd.read_csv(spacex_csv_file)

# Display the first few rows of the DataFrame to verify
print(spacex_df.head())


   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

Now, you can take a look at what are the coordinates for each site.


In [28]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [29]:
import folium
from folium.features import DivIcon

# Filter the required columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Create a base map centered around the first launch site
site_map = folium.Map(location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], zoom_start=10)

# Add markers and circles for each launch site
for _, row in launch_sites_df.iterrows():
    # Add a circle marker
    circle = folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#d35400',
        fill=True,
        fill_color='#d35400'
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Add a text marker
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    )
    
    # Add elements to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [30]:


# Add a marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through DataFrame and add markers
for _, row in spacex_df.iterrows():
    color = 'green' if row['class'] == 1 else 'red'
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Site: {row['Launch Site']}<br>Status: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)

# Display map
site_map

In [31]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='bottomright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [32]:
coast = [34.63683, -120.62461]
railway = [34.63684, -120.62364]
highway = [34.66666, -120.55555]
launch_site_lat = spacex_df.iloc[3]['Lat']
launch_site_lon = spacex_df.iloc[3]['Long']

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [33]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [34]:

# Step 2: Select a launch site
launch_site_lat = spacex_df.iloc[3]['Lat']
launch_site_lon = spacex_df.iloc[3]['Long']

coast = [34.63683, -120.62461]
railway = [34.63684, -120.62364]
highway = [34.66666, -120.55555]
launch_site_lat = spacex_df.iloc[26]['Lat']
launch_site_lon = spacex_df.iloc[26]['Long']

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coast[0], coast[1])
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway[0], railway[1])
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway[0], highway[1])

# Step 5: Add markers
# Step 5: Add markers for the launch site, coastline, railway, and highway
folium.Marker([launch_site_lat, launch_site_lon], popup="Launch Site", icon=folium.Icon(color="green")).add_to(site_map)
folium.Marker([coast[0], coast[1]], popup="Closest Coastline", icon=folium.Icon(color="blue")).add_to(site_map)
folium.Marker([railway[0], railway[1]], popup="Closest Railway", icon=folium.Icon(color="gray")).add_to(site_map)
folium.Marker([highway[0], highway[1]], popup="Closest Highway", icon=folium.Icon(color="black")).add_to(site_map)

# Step 6: Draw lines between the launch site and the points
folium.PolyLine([(launch_site_lat, launch_site_lon), (coast[0], coast[1])], color="blue", weight=2.5).add_to(site_map)
folium.PolyLine([(launch_site_lat, launch_site_lon), (railway[0], railway[1])], color="gray", weight=2.5).add_to(site_map)
folium.PolyLine([(launch_site_lat, launch_site_lon), (highway[0], highway[1])], color="black", weight=2.5).add_to(site_map)

# Step 7: Add distance labels for all points
for loc, distance, color in zip(
    [coast, railway, highway],
    [distance_coastline, distance_railway, distance_highway],
    ["#d35400", "#2c3e50", "#1abc9c"]
):
    folium.Marker(
        location=[loc[0], loc[1]],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:{color};"><b>{distance:.2f} KM</b></div>',
        )
    ).add_to(site_map)

# Display the map
site_map
